# Obtaining and Importing the Data

In [ ]:
%%bash

qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza
  
mv emp-single-end-sequences.qza ./emp-single-end-sequences/

In [ ]:
%%bash

qiime tools peek ./emp-single-end-sequences/emp-single-end-sequences.qza

## Demultiplexing

In [ ]:
%%bash

qiime demux emp-single \
  --i-seqs ./emp-single-end-sequences/emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences ./demultiplexed-data/demux.qza \
  --o-error-correction-details ./demultiplexed-data/demux-details.qza

sample-metadata.tsv holds the barcode of each sample needed for demultiplexing.

In [ ]:
%%bash

qiime demux summarize \
  --i-data ./demultiplexed-data/demux.qza \
  --o-visualization ./demultiplexed-data/demux.qzv

The file demux.qzv can be viewed at view.qiime2.org.

# Sequence quality control and feature table construction

## Option 1: DADA 2

In [19]:
%%bash

qiime dada2 denoise-single \
  --i-demultiplexed-seqs ./demultiplexed-data/demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences ./dada-2/rep-seqs-dada2.qza \
  --o-table ./dada-2/table-dada2.qza \
  --o-denoising-stats ./dada-2/stats-dada2.qza

Saved FeatureTable[Frequency] to: ./dada-2/table-dada2.qza
Saved FeatureData[Sequence] to: ./dada-2/rep-seqs-dada2.qza
Saved SampleData[DADA2Stats] to: ./dada-2/stats-dada2.qza


In [20]:
%%bash

qiime metadata tabulate \
  --m-input-file ./dada-2/stats-dada2.qza \
  --o-visualization ./dada-2/stats-dada2.qzv

Saved Visualization to: ./dada-2/stats-dada2.qzv


stats-dada2.qzv is a visualization file -> view it

## Option 2: Deblur

Filter the data

In [21]:
%%bash

qiime quality-filter q-score \
 --i-demux ./demultiplexed-data/demux.qza \
 --o-filtered-sequences ./deblur/demux-filtered.qza \
 --o-filter-stats ./deblur/demux-filter-stats.qza

Saved SampleData[SequencesWithQuality] to: ./deblur/demux-filtered.qza
Saved QualityFilterStats to: ./deblur/demux-filter-stats.qza


Denoise the data

In [22]:
%%bash

qiime deblur denoise-16S \
  --i-demultiplexed-seqs ./deblur/demux-filtered.qza \
  --p-trim-length 120 \
  --o-representative-sequences ./deblur/rep-seqs-deblur.qza \
  --o-table ./deblur/table-deblur.qza \
  --p-sample-stats \
  --o-stats ./deblur/deblur-stats.qza

Saved FeatureTable[Frequency] to: ./deblur/table-deblur.qza
Saved FeatureData[Sequence] to: ./deblur/rep-seqs-deblur.qza
Saved DeblurStats to: ./deblur/deblur-stats.qza


In [23]:
%%bash

qiime metadata tabulate \
  --m-input-file ./deblur/demux-filter-stats.qza \
  --o-visualization ./deblur/demux-filter-stats.qzv
qiime deblur visualize-stats \
  --i-deblur-stats ./deblur/deblur-stats.qza \
  --o-visualization ./deblur/deblur-stats.qzv

Saved Visualization to: ./deblur/demux-filter-stats.qzv
Saved Visualization to: ./deblur/deblur-stats.qzv


# FeatureTable and FeatureData summaries

In [24]:
%%bash

qiime feature-table summarize \
  --i-table ./deblur/table-deblur.qza \
  --o-visualization ./feature-summaries/table.qzv \
  --m-sample-metadata-file sample-metadata.tsv
qiime feature-table tabulate-seqs \
  --i-data ./deblur/rep-seqs-deblur.qza \
  --o-visualization ./feature-summaries/rep-seqs.qzv

Saved Visualization to: ./feature-summaries/table.qzv
Saved Visualization to: ./feature-summaries/rep-seqs.qzv


# Generate a tree for phylogenetic diversity analyses

In [25]:
%%bash

qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences ./deblur/rep-seqs-deblur.qza \
  --o-alignment ./pylogenetic-diversity-analysis/aligned-rep-seqs.qza \
  --o-masked-alignment ./pylogenetic-diversity-analysis/masked-aligned-rep-seqs.qza \
  --o-tree ./pylogenetic-diversity-analysis/unrooted-tree.qza \
  --o-rooted-tree ./pylogenetic-diversity-analysis/rooted-tree.qza

Saved FeatureData[AlignedSequence] to: ./pylogenetic-diversity-analysis/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: ./pylogenetic-diversity-analysis/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: ./pylogenetic-diversity-analysis/unrooted-tree.qza
Saved Phylogeny[Rooted] to: ./pylogenetic-diversity-analysis/rooted-tree.qza


# Alpha and beta diversity analysis

In [27]:
%%bash

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ./pylogenetic-diversity-analysis/rooted-tree.qza \
  --i-table ./deblur/table-deblur.qza \
  --p-sampling-depth 1103 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [28]:
%%bash

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


In [29]:
%%bash

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-body-site-significance.qzv
Saved Visualization to: core-metrics-results/unweighted-unifrac-subject-group-significance.qzv


In [30]:
%%bash

qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv

qiime emperor plot \
  --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/bray-curtis-emperor-days-since-experiment-start.qzv

Saved Visualization to: core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv
Saved Visualization to: core-metrics-results/bray-curtis-emperor-days-since-experiment-start.qzv


# Alpha rarefaction plotting

In [31]:
%%bash

qiime diversity alpha-rarefaction \
  --i-table ./deblur/table-deblur.qza \
  --i-phylogeny ./pylogenetic-diversity-analysis/rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization ./alpha-rarefaction/alpha-rarefaction.qzv

Saved Visualization to: ./alpha-rarefaction/alpha-rarefaction.qzv


# Taxonomic analysis

In [31]:
%%bash

qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

Plugin error from feature-classifier:

  The scikit-learn version (0.24.1) used to generate this artifact does not match the current version of scikit-learn installed (0.23.1). Please retrain your classifier for your current deployment to prevent data-corruption errors.

Debug info has been saved to /tmp/qiime2-q2cli-err-1gm252bj.log


CalledProcessError: Command 'b'\nqiime feature-classifier classify-sklearn \\\n  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \\\n  --i-reads rep-seqs.qza \\\n  --o-classification taxonomy.qza\n'' returned non-zero exit status 1.

In [ ]:
%%bash

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv